In [1]:
import modern_robotics as mr
import numpy as np
from calibration import *
np.set_printoptions(precision=16)

In [2]:
s = 0.001

w1 = np.array([0,0,1])
v1 = np.array([0,0,0])*s

w2 = np.array([0,-1,0])
v2 = np.array([0,0,0])*s

w3 = np.array([0,-1,0])
v3 = np.array([0,0,-100])*s

w4 = np.array([0,0,-1])
v4 = np.array([-50,250,0])*s

w5 = np.array([0,-1,0])
v5 = np.array([-20,0,-250])*s

w6 = np.array([0,0,-1])
v6 = np.array([-50,250,0])*s


xi0 = np.c_[np.r_[w1,v1], np.r_[w2,v2], np.r_[w3,v3], np.r_[w4,v4], np.r_[w5,v5], np.r_[w6,v6]]


wist0 = np.array([0,0,0])
vist0 = np.array([250,50,-20])*s

xist0 = np.r_[wist0, vist0]


# introducing some random error

xi01=np.array([0.0399,-0.0199,0.9989,0.0200,0.0400,0])
xi02=np.array([0,-1,0,-0.0200,0,0.0500])
xi03=np.array([0.1780,-0.9840,-0.0010,-0.0841,0.0874,-100.9999])
xi04=np.array([0.0619,0.0129,-0.9979,-50.9999,249.0000,0.0751])
xi05=np.array([0.0009,-0.9999,0,-20.6000,-0.0205,-249])
xi06=np.array([0.0949,0.0309,-0.9949,-51.2730,248.9109,2.8595])

xi00=np.c_[xi01,xi02,xi03,xi04,xi05,xi06]
xi00[3:6,:] = xi00[3:6,:]*s

#initial twist
xist00=np.array([0.02, -0.01, 0.01, 249, 51, -20.6])
xist00[3:6] = xist00[3:6]*s


mean_error_before = np.zeros((10,1))
mean_error_traditional = np.zeros((10,1))

max_error_before = np.zeros((10,1))
max_error_traditional = np.zeros((10,1))


In [3]:
for N in range(10, 110, 10): 
# for N in [10]:    # only run for one iteration (10 simulated measurements)

    P01 = np.array([100, 0, 0]) * s
    P02 = np.array([0, 100, 0]) * s
    P03 = np.array([0, 0, 100]) * s
    PX = np.c_[P01, P02, P03]

    P04 = np.array([-100, -100, -100]) * s
    
    gm = np.zeros((4, 4, N))
    gn = np.zeros((4, 4, N))
    Pa1 = np.zeros((4, N))
    Pa2 = np.zeros((4, N))
    Pa3 = np.zeros((4, N))
    M_home = np.eye(4)

    cali_joint_angles = np.random.rand(N,6) * 2 * np.pi - np.pi

    for i in range(0,N):
                
        Pa1[:, i] = np.dot(mr.FKinSpace(se3_exp(xist00), xi00, cali_joint_angles[i, :]), np.r_[P01, 1].reshape((4, 1)) ).reshape((4,)) + ((np.random.rand(4, 1) * 0.2 - 0.1) * s).reshape((4,)) # simulating inaccuracies in measurement [-0.1, 0.1] mm
        
        Pa2[:, i] = np.dot(mr.FKinSpace(se3_exp(xist00), xi00, cali_joint_angles[i, :]), np.r_[P02, 1].reshape((4, 1)) ).reshape((4,)) + ((np.random.rand(4, 1) * 0.2 - 0.1) * s).reshape((4,))
        
        Pa3[:, i] = np.dot(mr.FKinSpace(se3_exp(xist00), xi00, cali_joint_angles[i, :]), np.r_[P03, 1].reshape((4, 1)) ).reshape((4,)) + ((np.random.rand(4, 1) * 0.2 - 0.1) * s).reshape((4,))
        
        PY = np.c_[Pa1[0:3, i], Pa2[0:3, i], Pa3[0:3, i]]

        R, t = registration(PX, PY)

        gm[0:3, 0:3, i] = R
        gm[0:3, 3, i] = t
        gn[:, :, i] = mr.FKinSpace(se3_exp(xist00), xi00, cali_joint_angles[i, :])

    xiTFP, xistTFP = traditional_calibration_6dof(xi0, xist00, cali_joint_angles, gm, 10)

    Nt=100
    
    error_before=np.zeros((Nt,1))
        
    error_afterTraditional=np.zeros((Nt,1))
    
    test_joint_config = np.random.rand(Nt,1) * 2 * np.pi - np.pi

    
    for i in range(0,Nt):
        
        error_before[i] = np.linalg.norm(np.dot(mr.FKinSpace(se3_exp(xist0), xi0, test_joint_config[i, :]) - mr.FKinSpace(se3_exp(xist00) , xi00, test_joint_config[i, :]), np.r_[P04, 1])) / s
        
        error_afterTraditional[i] = np.linalg.norm(np.dot(mr.FKinSpace(se3_exp(xistTFP), xiTFP, test_joint_config[i, :]) - mr.FKinSpace(se3_exp(xist00), xi00, test_joint_config[i, :]), np.r_[P04, 1]) ) / s

    idx = int(N/10) - 1

    mean_error_before[idx, 0] = np.mean(error_before)
    mean_error_traditional[idx, 0] = np.mean(error_afterTraditional)


In [4]:
print("Mean error before: ", mean_error_before)
print("Mean error traditional: ", mean_error_traditional)

Mean error before:  [[9.326457505855657]
 [9.767852307771287]
 [8.897342156619665]
 [8.782603532497562]
 [9.122547032876687]
 [9.257873917730603]
 [8.827723812718169]
 [9.351311195182873]
 [9.275779570096681]
 [9.181916171340982]]
Mean error traditional:  [[0.1067619970331664]
 [0.1030563411213622]
 [0.132854308762433 ]
 [0.081681867412318 ]
 [0.0874713053235934]
 [0.066187506845226 ]
 [0.0388490697808006]
 [0.0764738187039574]
 [0.0574224279476038]
 [0.0489120401071146]]
